## Set-up

In [ ]:
# Environment setup
from google.colab import drive
import pandas as pd
import numpy as np
!pip3 install num2words
from num2words import num2words
import re
import os
import time
!pip3 install contractions
import contractions
import tensorflow as tf
import keras
!pip3 install transformers
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)
import torch
import torch.nn as nn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=711b73aeca5bc84c88ed090eeeb6bbec63c858afaf94676e8dcfa8de124621d6
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 8.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199

In [ ]:
# Mount GoogleDrive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load Original Text

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/SciFi/internet_archive_scifi_v3.txt','r', encoding="utf-8") as f:
  scifi = f.read()

num_chars = len(scifi)
print("Number of characters in the document:", num_chars)

Number of characters in the document: 149326361


In [ ]:
# The unique characters in the file
vocab = sorted(set(scifi))
print(f'{len(vocab)} unique characters')

75 unique characters


In [ ]:
# Extract the first 5500 characters of the row
first_5500_chars = scifi[:5500]
print(first_5500_chars) # Remove the first 5200 as copyright materical and not scifi stories.

MARCH # All Stories New and Complete Publisher Editor IF is published bi-monthly by Quinn Publishing Company, Inc., Kingston, New York. Volume #, No. #. Copyright # by Quinn Publishing Company, Inc. Application for Entry' as Second Class matter at Post Office, Buffalo, New York, pending. Subscription # for # issues in U.S. and Possessions: Canada # for # issues; elsewhere #. Aiiow four weeks for change of address. All stories appearing in this magazine are fiction. Any similarity to actual persons is coincidental. #c a fcopy. Printed ia U.S. A. A chat with the editor  i #  science fiction magazine called IF. The title was selected after much thought because of its brevity and on the theory it is indicative of the field and will be easy to remember. The tentative title that just morning and couldn't remember it until we'd had a cup of coffee, it was summarily discarded. A great deal of thought and effort lias gone into the formation of this magazine. We have had the aid of several very 

In [ ]:
# Find how many times 'pwf' appear
print(scifi.find('pwf')) # Remove

5196


In [ ]:
# Extract the last 2000 characters
last_2000_chars = scifi[-2000:]
print(last_2000_chars) # Remove the last 1944 as not scifi stories.

tury undersea mission from which no human has ever returned. Now, # feet down, you learn that an (Publ. ed. #) Ed. by T. E. Dikty # top-notch short stories and novelettes by such well known writers as Robert Bloch. Frank Robinson, Mark Clifton, many others. (Publ. ed. #) Just Mail Coupon In.ll.'i.l ' mi coupon your choice of waul in inhlit imi to "Satellite!" One bool, will lie considered your first i led ion for which you'll be billed only I plus a few cents jnon I I I you will be offered the cream of I lie new # to # science lid ion books -- for only I each. You take only those books you rcnlly want -- as few as four a year llut this offer may be with. I I a w I I at any time. So mail Del OX-#, Garden City, N. Y. Dept. GX-#, Garden City, N. Y. Enroll me as a member, and rush me my full-length handcated below. Bill me only I (plus few cents shipping charges). Every month send me the Club's free bulletin, "Things to Come," so that I may decide whether or not I wish to receive the coini

In [ ]:
# Define function to calculate average sentence length
def average_sentence_len(text):
  sentences = text.split(".")
  words = text.split(" ")
  if (sentences[len(sentences)-1]==""):
    average_sentence_length= len(words) / len(sentences)-1
  else:
    average_sentence_length = len(words) / len(sentences)

  return average_sentence_length

# Define function to calculate maximum sentence length
def max_sentence_len(text):
  sentences = text.split(".")
  max_len = max([len(sentence.split()) for sentence in sentences])
  return max_len

In [ ]:
# Get max and average sentence length
average_sentence_length = average_sentence_len(scifi)
max_sentence_length = max_sentence_len(scifi)
print(f"Average sentence length: {average_sentence_length}")
print(f"Maximum sentence length: {max_sentence_length}")

Average sentence length: 12.553956848085688
Maximum sentence length: 326


In [ ]:
# Count how many words are in the text
word_count = len(scifi.split(" "))
print("The number of words in the sample text is:", word_count)

The number of words in the sample text is: 26655101


## Preprocessing

In [ ]:
# Define a function to convert numbers to text
def convert_numbers_to_text(text):
    numbers = re.findall(r'\d+', text)
    for number in numbers:
        text = text.replace(number, num2words(int(number)))
    return text

In [ ]:
# Define function to clean and preprocess the text
def preprocess(text):
  text = text.replace(text[:5200], '') # Delete start of dataset
  text = text.replace(text[-1994:], '') # Delete end of dataset
  text = text.replace('pwf', '')
  text = convert_numbers_to_text(text)
  text = text.replace('&', 'and')
  text = text.replace('%', ' percent')
  text = contractions.fix(text)
  # A list of special charaters to be removed
  special_characters = ['@', '#', '$', '£', '*', '-', '+', '=', '~', '<', '>']
  for i in special_characters:
    if i != ',' or '.':
      text = text.replace(i, "")
  return text

In [ ]:
# Preprocess the original text
clean_scifi = preprocess(scifi)

In [ ]:
# Save the preprocessed text to a file
with open('/content/gdrive/MyDrive/Colab Notebooks/CSC8637/SciFi/clean_scifi.txt', 'w', encoding='utf-8') as f:
  f.write(clean_scifi)

In [ ]:
# Extract the first 300 characters to check it's correct
first_300_chars = clean_scifi[:300]
print(first_300_chars)

 Police grilled him mercilessly  while eyes from a hundred worlds looked on  It was a lovetriangle murder that made today's headlines but the answer lay hundreds of thousands of light years away! one of the basement rooms. He moved slowly and with a kind of painful dignity, as a man moves on his way


In [ ]:
# Get max and average sentence length
average_sentence_length = average_sentence_len(clean_scifi)
max_sentence_length = max_sentence_len(clean_scifi)
print(f"Average sentence length: {average_sentence_length}")
print(f"Maximum sentence length: {max_sentence_length}")

Average sentence length: 12.755057726536492
Maximum sentence length: 321


In [ ]:
# The unique characters in the file
clean_vocab = sorted(set(clean_scifi))
print(f'{len(clean_vocab)} unique characters')

63 unique characters


In [ ]:
# Count how many words are in the text
clean_word_count = len(clean_scifi.split(" "))
print("The number of words in the text is:", clean_word_count)

The number of words in the text is: 27073809


## Load Preprocessed Text: Tokenizer and DataCollator

In [ ]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
# Check the tokenizer works as expected
print('vocabulary size: %d, max squence length: %d' % (tokenizer.vocab_size, tokenizer.model_max_length))
print('tokenize sequence "I was smart enough to fool all the so-called brains of the Solar System.":', tokenizer('I was smart enough to fool all the so-called brains of the Solar System.'))

vocabulary size: 50257, max squence length: 1024
tokenize sequence "I was smart enough to fool all the so-called brains of the Solar System.": {'input_ids': [40, 373, 4451, 1576, 284, 9192, 477, 262, 523, 12, 7174, 14290, 286, 262, 12347, 4482, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
# Load in pre-processed text with tokenizer
text = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/gdrive/MyDrive/Colab Notebooks/CSC8637/SciFi/clean_scifi.txt',
    block_size=128)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# Check it worked
print(tokenizer.decode(text[5]))

 used would be a terrific hammer. The tack he drove would outshade other tacks by five country miles. And the picture? Gad, what a masterpiece! Seriously, one has only fo view Browne's enthusiasm for living to know it for what it is  a priceless gift. He has written unnumbered short stories and, under the name of John Evans, is the father of near future. We have watched him write several of his stories and he hurled himself into each with a zeal and a zest that sfunned us into a partial paralysis. So we give you Howard Browne, a hard fellow to classify ; an astounding mixture


In [ ]:
# Load a DataCollector to form batches from text
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Modelling

In [ ]:
# Set up model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Add two RNN layers
model.resize_token_embeddings(len(tokenizer))
model.output_hidden_states = True
model.lstm1 = torch.nn.LSTM(input_size=model.config.hidden_size, hidden_size=512, num_layers=1, batch_first=True)
model.lstm2 = torch.nn.LSTM(input_size=512, hidden_size=256, num_layers=1, batch_first=True)
model.fc = torch.nn.Linear(in_features=256, out_features=len(tokenizer))

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir = '/content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_adjustme', # the output directory for the model predictions and checkpoints
    overwrite_output_dir = True, # overwrite the content of the output directory
    per_device_train_batch_size = 32, # the batch size for training
    per_device_eval_batch_size = 32, # the batch size for evaluation
    learning_rate = 5e-5, # defaults to 5e-5
    num_train_epochs = 3, # total number of training epochs to perform
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Define the training parameters
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = text
)

In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 268819
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 25203
  Number of trainable parameters = 140769873


Step,Training Loss
500,3.723900
1000,3.717300
1500,3.710400
2000,3.713800
2500,3.714900
3000,3.712900
3500,3.710100
4000,3.715600
4500,3.715100
5000,3.713700


Saving model checkpoint to /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-500
Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-500/config.json
Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-500/generation_config.json
Model weights saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-1000
Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-1000/config.json
Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-1000/generation_config.json
Model weights saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/gdrive/MyDrive/Colab N

TrainOutput(global_step=25203, training_loss=3.7043542712471607, metrics={'train_runtime': 6525.5237, 'train_samples_per_second': 123.585, 'train_steps_per_second': 3.862, 'total_flos': 6.279436939906944e+16, 'train_loss': 3.7043542712471607, 'epoch': 3.0})

In [ ]:
# Save the model
trainer.save_model()

Saving model checkpoint to /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5
Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/config.json
Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/generation_config.json
Model weights saved in /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2_2e-5/pytorch_model.bin


## Generate Text

In [ ]:
# Create the generator
generator = pipeline('text-generation', tokenizer='gpt2', model='/content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2')

Some weights of the model checkpoint at /content/gdrive/MyDrive/Colab Notebooks/CSC8637/models/gpt2 were not used when initializing GPT2LMHeadModel: ['lstm1.bias_hh_l0', 'fc.bias', 'lstm2.bias_hh_l0', 'lstm1.weight_hh_l0', 'lstm2.weight_ih_l0', 'lstm1.weight_ih_l0', 'lstm2.bias_ih_l0', 'fc.weight', 'lstm1.bias_ih_l0', 'lstm2.weight_hh_l0']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Example prompts to be inserted in ' ' below:
#My only thought was to hide
#She shook her head emphatically
#handle it wisely
#At first, of course, you will rebel.
#I was smart enough to fool all the so-called brains of the Solar System
#They should never have trusted me.

print(generator("""If you missed the train I'm on
You will know that I am gone
You can hear the whistle blow a hundred miles
A hundred miles, a hundred miles
A hundred miles, a hundred miles
You can hear the whistle blow a hundred miles
Lord, I'm one, Lord, I'm two
Lord, I'm three, Lord, I'm four
…""", max_length=200)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


If you missed the train I'm on
You will know that I am gone
You can hear the whistle blow a hundred miles
A hundred miles, a hundred miles
A hundred miles, a hundred miles
You can hear the whistle blow a hundred miles
Lord, I'm one, Lord, I'm two
Lord, I'm three, Lord, I'm four
… and you know it and you are a different thing than I am." His cheeks had begun bleeding. "Lord, what can I do? I never knew how to make love and I never knew how to make love." "You have a girl with you that you want to have. You do not have to know it or know why it is. You are not a little girl and the way the boy wants to have is you, the way he thinks you want to have." "What about the girl?" "She wants to have me," said Grom, "we are going somewhere and the other day


**The llamas stole all my honey.** I told them." It took several months for the llamas to show up alive in Luna City, but they did. During each of those months, the old Llama was still around, waiting by the river for the sun to set. The river was not hot enough to swim and the Llama did not mind the sight, even if it meant drowning. The first Llamar had caught a wind to go on a run, and had pulled itself into a shallow pool where the water ran deep into a black crevice. The old Llamar had lived a hundred and fifty years at one of the few lakes that were still alive. The great, deeplung black lake had given birth to a pair of longlegged Llamaras, each a pair of wings. At first there seemed to be little difference in appearance to the giant reptiles, but the Llairs had developed to a form.

**At first, of course, you will rebel.** But after a few days of intensive training, you will realize that it is not only a question of fighting, but of gaining experience to give yourself a new tool to fight." He was a very young man, but he thought of this, thought of its value as an opportunity. He wished he had a new idea. "To have a chance, if you will excuse me, to go up and put us in touch with some of the best men ever to make interstellar space flight possible." "Thank you, sir." The ambassador was very angry. He waved and strode away. A few minutes later, the ship came to a landing on a blackened, empty world. From it he could see four of the crewmen floating in the dimlylit room beneath a high, dark dome below the window. The man standing in the doorway said: "There is a message waiting for me." "Ah. There it is."